# Train models

With this notebook, we train simple untunted recurrent models with sufficient performance to demonstrate the effect of data augmentation strategies

In [ ]:
import os, time, pickle, boto3, sys
from datetime import datetime

import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, average_precision_score, roc_curve, auc, confusion_matrix

In [ ]:
from utils.datagen import PROCESSED_DATAPATH, MODEL_INPUT_DATAPATH, RESULTFILE_DATAPATH, DataGen, strategies, targets, get_datafile
from utils.utils import read_data, dump_data
from utils.connections import model_input_bucket, processed_data_bucket, get_s3_keys_as_generator, download_file

# check pre-reqs. 
assert(tf.__version__[0]=='2')
assert len(tf.config.experimental.list_physical_devices('GPU')) > 0

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
def create_model(max_features, output_dim, num_units, name, timesteps=500, dropout=0.3, unit_type='LSTM'):
    model_input = tfk.Input(shape=(timesteps,), name=f'{name}_input')
    x = tfkl.Embedding(max_features+1, output_dim, name=f'{name}_embed')(model_input)
    if unit_type == 'LSTM':
        x = tfkl.Bidirectional(tfkl.LSTM(units=num_units, name=f'{name}_recurrent',
                                         return_sequences=False, 
                                         kernel_initializer='glorot_uniform', 
                                         bias_initializer='zeros'))(x)
    elif unit_type == 'GRU':
        x = tfkl.Bidirectional(tfkl.GRU(units=num_units, name=f'{name}_recurrent',
                                        return_sequences=False, 
                                        kernel_initializer='glorot_uniform', 
                                        bias_initializer='zeros'))(x)
    else:
        raise NotImplementedError
    x = tfkl.LeakyReLU(alpha=0.3)(x)
    x = tfkl.Dropout(dropout)(x)
    x = tfkl.Dense(num_units//2, activation='relu')(x)
    x = tfkl.Dropout(dropout)(x)
    model_output = tfkl.Dense(2, activation='softmax', name=f'{name}_output')(x)
    return tfk.Model(inputs=model_input, outputs=model_output)

In [ ]:
import pickle

def valid_epoch(data_valid, model, valid_step, data_index, valid_output=''):
    valid_iter = iter(data_valid)
    preds = []
    ys = []
    id_list = []
    valid_loss = []
    while True:
        try:
            x_clin, x_vs, y, ids, tt = next(valid_iter)
            x = [x_clin, x_vs]
            if data_index >= 0:
                x = x[data_index]
            pred, loss = valid_step(x, y, model)
            valid_loss.append(loss.numpy())
            preds.append(pred.numpy())
            ys.append(y.numpy())
            id_list.append(ids.numpy())
        except:# StopIteration:
            break
    probabilities = np.vstack(preds)[:,0]
    Y = np.hstack(ys)
    IDS = np.hstack(id_list)
    predictions = np.where(probabilities < 0.5, 0., 1.)
    
    if valid_output != '':
        with open(valid_output, 'wb+') as outfile:
            pickle.dump(probabilities, outfile)
            pickle.dump(Y, outfile)
            pickle.dump(IDS, outfile)
    tp = len(np.where((predictions==Y)&(predictions==1))[0])
    fp = len(np.where((predictions!=Y)&(predictions==1))[0])
    tn = len(np.where((predictions==Y)&(predictions==0))[0])
    fn = len(np.where((predictions!=Y)&(predictions==0))[0])
    accuracy = (tp + tn)/len(predictions)
    fpr, tpr, thresholds = roc_curve(Y, probabilities)
    val_auc = auc(fpr, tpr)
    sensitivity = tp/(tp + fn)
    specificity = tn/(tn + fp)
    try:
        WDR = (tp + fp)/tp
    except ZeroDivisionError:
        WDR = 0
    return np.mean(valid_loss), tp, fp, tn, fn, accuracy, val_auc, sensitivity, specificity, WDR

def train_epoch(data_train, model, train_step, data_index):
    t0 = time.clock()
    train_iter = iter(data_train)
    epoch_losses = []
    while True:
        try:
            x_clin, x_vs, y, ids, tt = next(train_iter)
            _, loss = train_step(x_clin, y, clin_model)
            epoch_losses.append(loss.numpy())
        except:# StopIteration:
            break
    t1 = time.clock() - t0
    return t1, np.mean(epoch_losses)

In [ ]:
def train_model(data_train, data_valid, model, data_index, epochs, label, epoch_start=0):
    result_strings = []
    @tf.function
    def get_loss(Y, predictions):
        return tfk.backend.binary_crossentropy(predictions[:,0], tf.cast(Y, tf.float32))

    @tf.function
    def train_step(x, y, model):
        with tf.GradientTape() as tape:
            preds = model(x, training=True)
            loss = tf.reduce_mean(get_loss(y, preds))
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return preds, loss

    @tf.function
    def valid_step(x, y, model, training=False):
        preds = model(x, training=training)
        loss = tf.reduce_mean(get_loss(y, preds))
        return preds, loss

    runfile_name = datetime.now().strftime("%Y%m%d_%H:%M:%S")
    losses = []
    min_loss = 1e20
    for e in range(epoch_start, epochs):
        elapsed_time, epoch_loss = train_epoch(data_train, model, train_step, data_index)
        valid_output = f'{label}_{e}_preds'
        val_loss, tp, fp, tn, fn, accuracy, val_auc, sensitivity, specificity, WDR = valid_epoch(data_valid, model, valid_step, data_index, valid_output)
        result_string = (f'{label}\t{e}\t{elapsed_time}\t{val_loss}\t{epoch_loss}\t{tp}\t{fp}\t{tn}\t{fn}\t{accuracy}\t{val_auc}\t{sensitivity}\t{specificity}\t{WDR}')
        print(result_string)
        result_strings.append(result_string)
        model.save_weights(os.path.join(RESULTFILE_DATAPATH, f'{label}_e{e}.h5'))
        with open(os.path.join(RESULTFILE_DATAPATH, 'summaries', f'{label}_{runfile_name}.tsv'), 'a+') as outfile:
            outfile.write(result_string + '\n')
        if val_loss < min_loss:
            min_loss = val_loss
            losses = []
        else:
            losses.append(val_loss)
        if len(losses)>3:
            break
    return result_strings

In [ ]:
batch_size=128

max_features = {'clin': 7916, 'vs': 19405}
embed_dim = {'clin': 150, 'vs': 100}
data_index = {'clin': 0, 'vs': 1}
widths = [5,10,15]
name = 'clin'
epochs = 100

for fold in range(5):
    for unit_type in ['GRU', 'LSTM']:
        for target in ['hosp_death', 'icu_death', 'long_icu']:
            for strategy in strategies:
                data_train = get_datafile(target, strategy, fold=fold, 
                                          phase='train', batch_size=batch_size, 
                                          model_type='both')
                data_valid = get_datafile(target, strategy, fold=fold, 
                                          phase='valid', batch_size=batch_size, 
                                          model_type='both')

                for model_width in widths:
                    clin_model = create_model(max_features[name], embed_dim[name], model_width, name, unit_type=unit_type)

                    try:
                        weight_files = [f for f in os.listdir(RESULTFILE_DATAPATH) if strategy in f and target in f and unit_type in f and f'width_{model_width}_fold_{fold}' in f and 'h5' in f]
                        completed_epochs = max([int(w.split('.')[0].split('_')[-1].strip('e')) for w in weight_files])
                        target_weight_file = f'{unit_type}_{strategy}_{target}_width_{model_width}_fold_{fold}_e{completed_epochs}.h5'
                        print(target_weight_file, target_weight_file in weight_files)    
                        clin_model.load_weights(os.path.join(RESULTFILE_DATAPATH, target_weight_file))
                        print(completed_epochs)
                    except (NameError, ValueError):
                        weight_files = []
                        completed_epochs = 0

                    optimizer = tfk.optimizers.Adam(learning_rate=1e-4, clipnorm=1)
                    clin_results = train_model(data_train, data_valid, clin_model, data_index[name], epochs, f'{unit_type}_{strategy}_{target}_width_{model_width}_fold_{fold}', completed_epochs + 1)